#Trump Tweet Analysis, Topic Modeling and Prediction of Energy Stock Prices

In [2]:
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification, pipeline, evaluation
from pyspark.sql import functions as fn, Row
from pyspark.sql.functions import when, regexp_extract, col
from pyspark import sql
from pyspark.sql.functions import *

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorIndexer

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
import re

In [3]:
#Importing the tweet file and converting it to a Data frame

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

trumpTweet_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true').option("delimiter", ",").option("escape", "\"").load('/FileStore/tables/trumptweets_2-a866d.csv')

In [4]:
#Converting the dataframe in pandas

trumpTweet_copy_df = trumpTweet_df
trumpTweet_pandas = trumpTweet_copy_df.toPandas()

##Initial Data Cleaning

In [6]:
#Removing special characters
trumpTweet_pandas['content'] = trumpTweet_pandas['content'].apply(lambda x: re.split('https:\/\/*', str(x))[0])

In [7]:
#Replacing blanks with 'nan'
trumpTweet_pandas['content'].replace('', np.nan, inplace=True)

In [8]:
trumpTweet_pandas.dropna(subset=['content'], inplace=True)

In [9]:
trumpTweet_pandas['content'] = trumpTweet_pandas['content'].apply(lambda x: re.split('http:\/\/*', str(x))[0])

In [10]:
#Grouping the tweets according to specific dates as we would merge this dataframe later on with stock prices per date

trumpTweet_groupedDate = trumpTweet_pandas.groupby(['date'])['content'].apply('-'.join).reset_index()

In [11]:
display(trumpTweet_groupedDate)

date,content
1/1/18,"HAPPY NEW YEAR! We are MAKING AMERICA GREAT AGAIN, and much faster than anyone thought possible!-The United States has foolishly given Pakistan more than 33 billion dollars in aid over the last 15 years, and they have given us nothing but lies & deceit, thinking of our leaders as fools. They give safe haven to the terrorists we hunt in Afghanistan, with little help. No more!-Iran is failing at every level despite the terrible deal made with them by the Obama Administration. The great Iranian people have been repressed for many years. They are hungry for food & for freedom. Along with human rights, the wealth of Iran is being looted. TIME FOR CHANGE!-Will be leaving Florida for Washington (D.C.) today at 4:00 P.M. Much work to be done, but it will be a great New Year!"
1/1/19,"HAPPY NEW YEAR!pic.twitter.com/bHoPDPQ7G6-MEXICO IS PAYING FOR THE WALL through the many billions of dollars a year that the U.S.A. is saving through the new Trade Deal, the USMCA, that will replace the horrendous NAFTA Trade Deal, which has so badly hurt our Country. Mexico & Canada will also thrive - good for all!-The Democrats will probably submit a Bill, being cute as always, which gives everything away but gives NOTHING to Border Security, namely the Wall. You see, without the Wall there can be no Border Security - the Tech “stuff” is just, by comparison, meaningless bells & whistles...-...Remember this. Throughout the ages some things NEVER get better and NEVER change. You have Walls and you have Wheels. It was ALWAYS that way and it will ALWAYS be that way! Please explain to the Democrats that there can NEVER be a replacement for a good old fashioned WALL!-Dr. Sebastian Gorka, a very good and talented guy, has a great new book just out, “Why We Fight.” Lots of insight - Enjoy!-HAPPY NEW YEAR TO EVERYONE, INCLUDING THE HATERS AND THE FAKE NEWS MEDIA! 2019 WILL BE A FANTASTIC YEAR FOR THOSE NOT SUFFERING FROM TRUMP DERANGEMENT SYNDROME. JUST CALM DOWN AND ENJOY THE RIDE, GREAT THINGS ARE HAPPENING FOR OUR COUNTRY!-Happy New Year!-The Democrats, much as I suspected, have allocated no money for a new Wall. So imaginative! The problem is, without a Wall there can be no real Border Security - and our Country must finally have a Strong and Secure Southern Border!-“General” McChrystal got fired like a dog by Obama. Last assignment a total bust. Known for big, dumb mouth. Hillary lover!-One thing has now been proven. The Democrats do not care about Open Borders and all of the crime and drugs that Open Borders bring!-Congratulations to President @ JairBolsonaro who just made a great inauguration speech - the U.S.A. is with you!-Border Security and the Wall “thing” and Shutdown is not where Nancy Pelosi wanted to start her tenure as Speaker! Let’s make a deal?-Gas prices are low and expected to go down this year. This would be good!-Washington Examiner - “MAGA list: 205 ‘historic results’ help Trump make case for 2020 re-election.” True!"
1/1/20,"Get this straightened out, Governor @ GavinNewsom-Wonderful account of U.S. Embassy (Iraq) vs. the Benghazi disaster!-How is the Paris Accord doing? Don’t ask!-One of my greatest honors was to have gotten CHOICE approved for our great Veterans. Others have tried for decades, and failed!-Thank you to the @ dcexaminer Washington Examiner. The list is growing every day!-Thank you Steve. The greatest Witch Hunt in U.S. history!-Our fantastic First Lady!-HAPPY NEW YEAR!-pic.twitter.com/EVAEYD1AgV"
1/10/18,"Today, it was my great honor to sign a new Executive Order to ensure Veterans have the resources they need as they transition back to civilian life. We must ensure that our HEROES are given the care and support they so richly deserve! -As I made very clear today, our country needs the security of the Wall on the Southern Border, which must be part of any DACA approval.-Thank you @ GOPLeader Kevin McCarthy! Couldn’t agree w/you more. TOGETHER, we are # MAGA-. @ ICEgov HSI agents and ERO officers, on 

In [12]:
import string

In [13]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

trumpTweet_groupedDate['cleaned_content'] = trumpTweet_groupedDate['content'].apply(lambda x: remove_punct(x))
trumpTweet_groupedDate.head(10)

,date,content,cleaned_content
0,1/1/18,HAPPY NEW YEAR! We are MAKING AMERICA GREAT AG...,HAPPY NEW YEAR We are MAKING AMERICA GREAT AGA...
1,1/1/19,HAPPY NEW YEAR!pic.twitter.com/bHoPDPQ7G6-MEXI...,HAPPY NEW YEARpictwittercombHoPDPQGMEXICO IS P...
2,1/1/20,"Get this straightened out, Governor @ GavinNew...",Get this straightened out Governor GavinNewso...
3,1/10/18,"Today, it was my great honor to sign a new Exe...",Today it was my great honor to sign a new Exec...
4,1/10/19,The Mainstream Media has NEVER been more disho...,The Mainstream Media has NEVER been more disho...
5,1/10/20,"THANK YOU TOLEDO, OHIO!-Under my administratio...",THANK YOU TOLEDO OHIOUnder my administration w...
6,1/11/18,The United States needs the security of the Wa...,The United States needs the security of the Wa...
7,1/11/19,Dear Diary...-Will be interviewed at the Borde...,Dear DiaryWill be interviewed at the Border by...
8,1/11/20,Will be interviewed tonight by Laura @ Ingraha...,Will be interviewed tonight by Laura Ingraham...
9,1/12/18,"Yesterday, I signed the # INTERDICTAct (H.R. 2...",Yesterday I signed the INTERDICTAct HR with ...


In [14]:
#display(trumpTweet_groupedDate)

In [15]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

trumpTweet_groupedDate['Tweet_tokenized'] = trumpTweet_groupedDate['cleaned_content'].apply(lambda x: tokenization(x.lower()))
#display(trumpTweet_groupedDate)

##Data Preprocessing (getting texts ready for topic modeling)

In [17]:

%sh 
pip install nltk
pip install --upgrade pip
python -m nltk.downloader all

Collecting nltk
 Downloading https://files.pythonhosted.org/packages/92/75/ce35194d8e3022203cca0d2f896dbb88689f9b3fce8e9f9cff942913519d/nltk-3.5.zip (1.4MB)
Requirement already satisfied: click in /databricks/python3/lib/python3.7/site-packages (from nltk) (7.1.2)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.7/site-packages (from nltk) (0.14.1)
Collecting regex (from nltk)
 Downloading https://files.pythonhosted.org/packages/0f/fa/a9e3ad47c189969e3233031187b323fe20005fee4bfc8edcb24f8ffbe540/regex-2020.4.4-cp37-cp37m-manylinux2010_x86_64.whl (679kB)
Requirement already satisfied: tqdm in /databricks/python3/lib/python3.7/site-packages (from nltk) (4.45.0)
Building wheels for collected packages: nltk
 Building wheel for nltk (setup.py): started
 Building wheel for nltk (setup.py): finished with status 'done'
 Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
Installing collected packages: regex, nltk
Successfully installed nltk-3.5 regex-2020.4.4
You are using pip version 19.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Collecting pip
 Downloading https://files.pythonhosted.org/packages/54/2e/df11ea7e23e7e761d484ed3740285a34e38548cf2bad2bed3dd5768ec8b9/pip-20.1-py2.py3-none-any.whl (1.5MB)
Installing collected packages: pip
 Found existing installation: pip 19.0.3
 Uninstalling pip-19.0.3:
 Successfully uninstalled pip-19.0.3
Successfully installed pip-20.1
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
 warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data] | 
[nltk_data] | Downloading package abc to /root/nltk_data...
[nltk_data] | Unzipping corpora/abc.zip.
[nltk_data] | Downloading package alpino to /root/nltk_data...
[nltk_data] | Unzipping corpora/alpino.zip.
[nltk_data] | Downloading package biocreative_ppi to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/biocreative_ppi.zip.
[nltk_data] | Downloading package brown to /root/nltk_data...
[nltk_data] | Unzipping corpora/brown.zip.
[nltk_data] | Downloading package brown_tei to /root/nltk_data...
[nltk_data] | Unzipping corpora/brown_tei.zip.
[nltk_data] | Downloading package cess_cat to /root/nltk_data...
[nltk_data] | Unzipping corpora/cess_cat.zip.
[nltk_data] | Downloading package cess_esp to /root/nltk_data...
[nltk_data] | Unzipping corpora/cess_esp.zip.
[nltk_data] | Downloading package chat80 to /root/nltk_data...
[nltk_data] | Unzipping corpora/chat80.zip.
[nltk_data] | Downloading package city_database to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/city_database.zip.
[nltk_data] | Downloading package cmudict to /root/nltk_data...
[nltk_data] | Unzipping corpora/cmudict.zip.
[nltk_data] | Downloading package comparative_sentences to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/comparative_sentences.zip.
[nltk_data] | Downloading package comtrans to /root/nltk_data...
[nltk_data] | Downloading package conll2000 to /root/nltk_data...
[nltk_data] | Unzipping corpora/conll2000.zip.
[nltk_data] | Downloading package conll2002 to /root/nltk_data...
[nltk_data] | Unzipping corpora/conll2002.zip.
[nltk_data] | Downloading package conll2007 to /root/nltk_data...
[nltk_data] | Downloading package crubadan to /root/nltk_data...
[nltk_data] | Unzipping corpora/crubadan.zip.
[nltk_data] | Downloading package dependency_treebank to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/dependency_treebank.zip.
[nltk_data] | Downloading package dolch to /root/nltk_data...
[nltk_data] | Unzipping corpora/dolch.zip.
[nltk_data] | Downloading package europarl_raw to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/europarl_raw.zip.
[nltk_data] |

In [18]:
import nltk

In [19]:
%sh pip install 
python -m spacy download en

ERROR: You must give at least one requirement to install (see "pip help install")
Requirement already satisfied: en_core_web_sm==2.2.5 from https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.5/en_core_web_sm-2.2.5.tar.gz#egg=en_core_web_sm==2.2.5 in /databricks/python3/lib/python3.7/site-packages (2.2.5)
Requirement already satisfied: spacy>=2.2.2 in /databricks/python3/lib/python3.7/site-packages (from en_core_web_sm==2.2.5) (2.2.4)
Requirement already satisfied: requests<3.0.0,>=2.13.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (2.21.0)
Requirement already satisfied: cymem<2.1.0,>=2.0.2 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (2.0.3)
Requirement already satisfied: plac<1.2.0,>=0.9.6 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (1.1.3)
Requirement already satisfied: preshed<3.1.0,>=3.0.2 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (3.0.2)
Requirement already satisfied: tqdm<5.0.0,>=4.38.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (4.45.0)
Requirement already satisfied: murmurhash<1.1.0,>=0.28.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (1.0.2)
Requirement already satisfied: thinc==7.4.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (7.4.0)
Requirement already satisfied: numpy>=1.15.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (1.16.2)
Requirement already satisfied: catalogue<1.1.0,>=0.0.7 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (1.0.0)
Requirement already satisfied: srsly<1.1.0,>=1.0.2 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (1.0.2)
Requirement already satisfied: blis<0.5.0,>=0.4.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (0.4.1)
Requirement already satisfied: wasabi<1.1.0,>=0.4.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (0.6.0)
Requirement already satisfied: setuptools in /usr/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (40.8.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests<3.0.0,>=2.13.0->spacy>=2.2.2->en_core_web_sm==2.2.5) (2019.3.9)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests<3.0.0,>=2.13.0->spacy>=2.2.2->en_core_web_sm==2.2.5) (2.8)
Requirement already satisfied: urllib3<1.25,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests<3.0.0,>=2.13.0->spacy>=2.2.2->en_core_web_sm==2.2.5) (1.24.1)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /databricks/python3/lib/python3.7/site-packages (from requests<3.0.0,>=2.13.0->spacy>=2.2.2->en_core_web_sm==2.2.5) (3.0.4)
Requirement already satisfied: importlib-metadata>=0.20; python_version < "3.8" in /databricks/python3/lib/python3.7/site-packages (from catalogue<1.1.0,>=0.0.7->spacy>=2.2.2->en_core_web_sm==2.2.5) (1.6.0)
Requirement already satisfied: zipp>=0.5 in /databricks/python3/lib/python3.7/site-packages (from importlib-metadata>=0.20; python_version < "3.8"->catalogue<1.1.0,>=0.0.7->spacy>=2.2.2->en_core_web_sm==2.2.5) (3.1.0)
 ✔ Download and installation successful 
You can now load the model via spacy.load('en_core_web_sm')
 ✔ Linking successful 
/databricks/python3/lib/python3.7/site-packages/en_core_web_sm -->
/databricks/python3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')

####Removing Stopwords

In [21]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
trumpTweet_groupedDate['removed_stopwords'] = trumpTweet_groupedDate['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))
#display(trumpTweet_groupedDate)

In [22]:

%sh 
pip install -U gensim


Collecting gensim
 Downloading gensim-3.8.2-cp37-cp37m-manylinux1_x86_64.whl (24.2 MB)
Requirement already satisfied, skipping upgrade: scipy>=1.0.0 in /databricks/python3/lib/python3.7/site-packages (from gensim) (1.2.1)
Requirement already satisfied, skipping upgrade: numpy>=1.11.3 in /databricks/python3/lib/python3.7/site-packages (from gensim) (1.16.2)
Collecting smart-open>=1.8.1
 Downloading smart_open-2.0.0.tar.gz (103 kB)
Requirement already satisfied, skipping upgrade: six>=1.5.0 in /databricks/python3/lib/python3.7/site-packages (from gensim) (1.12.0)
Requirement already satisfied, skipping upgrade: requests in /databricks/python3/lib/python3.7/site-packages (from smart-open>=1.8.1->gensim) (2.21.0)
Requirement already satisfied, skipping upgrade: boto in /databricks/python3/lib/python3.7/site-packages (from smart-open>=1.8.1->gensim) (2.49.0)
Requirement already satisfied, skipping upgrade: boto3 in /databricks/python3/lib/python3.7/site-packages (from smart-open>=1.8.1->gensim) (1.9.162)
Requirement already satisfied, skipping upgrade: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests->smart-open>=1.8.1->gensim) (2019.3.9)
Requirement already satisfied, skipping upgrade: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests->smart-open>=1.8.1->gensim) (2.8)
Requirement already satisfied, skipping upgrade: urllib3<1.25,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests->smart-open>=1.8.1->gensim) (1.24.1)
Requirement already satisfied, skipping upgrade: chardet<3.1.0,>=3.0.2 in /databricks/python3/lib/python3.7/site-packages (from requests->smart-open>=1.8.1->gensim) (3.0.4)
Requirement already satisfied, skipping upgrade: botocore<1.13.0,>=1.12.162 in /databricks/python3/lib/python3.7/site-packages (from boto3->smart-open>=1.8.1->gensim) (1.12.163)
Requirement already satisfied, skipping upgrade: jmespath<1.0.0,>=0.7.1 in /databricks/python3/lib/python3.7/site-packages (from boto3->smart-open>=1.8.1->gensim) (0.9.4)
Requirement already satisfied, skipping upgrade: s3transfer<0.3.0,>=0.2.0 in /databricks/python3/lib/python3.7/site-packages (from boto3->smart-open>=1.8.1->gensim) (0.2.1)
Requirement already satisfied, skipping upgrade: docutils>=0.10 in /databricks/python3/lib/python3.7/site-packages (from botocore<1.13.0,>=1.12.162->boto3->smart-open>=1.8.1->gensim) (0.14)
Requirement already satisfied, skipping upgrade: python-dateutil<3.0.0,>=2.1; python_version >= "2.7" in /databricks/python3/lib/python3.7/site-packages (from botocore<1.13.0,>=1.12.162->boto3->smart-open>=1.8.1->gensim) (2.8.0)
Building wheels for collected packages: smart-open
 Building wheel for smart-open (setup.py): started
 Building wheel for smart-open (setup.py): finished with status 'done'
 Created wheel for smart-open: filename=smart_open-2.0.0-py3-none-any.whl size=101341 sha256=cb851beb96e821ac84ba8291f6760bc2cf08980c1ea2905b4e48877fc2d9a65a
 Stored in directory: /root/.cache/pip/wheels/bb/1c/9c/412ec03f6d5ac7d41f4b965bde3fc0d1bd201da5ba3e2636de
Successfully built smart-open
Installing collected packages: smart-open, gensim
Successfully installed gensim-3.8.2 smart-open-2.0.0

In [23]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

/databricks/python/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
 _deprecated()
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Package wordnet is already up-to-date!
Out[146]: True

In [24]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Package wordnet is already up-to-date!
Out[147]: True

####Lemmatization

In [26]:
ps = nltk.PorterStemmer()
def lemmatize_stemming(text):
  word = WordNetLemmatizer().lemmatize(text, pos='v')
  #print(word)
  stemmed_word = nltk.PorterStemmer.stem(word)
  print(stemmed_word)
  return stemmed_word
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
      if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
        result.append(lemmatize_stemming(token))
        return result

In [27]:
#trumpTweet_groupedDate['preprocessed_text'] = trumpTweet_groupedDate['cleaned_content'].apply(lambda x: preprocess(x))

####Stemming

In [29]:
ps = nltk.snowball.EnglishStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

trumpTweet_groupedDate['stemmed_words'] = trumpTweet_groupedDate['removed_stopwords'].apply(lambda x: stemming(x))
trumpTweet_groupedDate.head()

,date,content,cleaned_content,Tweet_tokenized,removed_stopwords,stemmed_words
0,1/1/18,HAPPY NEW YEAR! We are MAKING AMERICA GREAT AG...,HAPPY NEW YEAR We are MAKING AMERICA GREAT AGA...,"[happy, new, year, we, are, making, america, g...","[happy, new, year, making, america, great, muc...","[happi, new, year, make, america, great, much,..."
1,1/1/19,HAPPY NEW YEAR!pic.twitter.com/bHoPDPQ7G6-MEXI...,HAPPY NEW YEARpictwittercombHoPDPQGMEXICO IS P...,"[happy, new, yearpictwittercombhopdpqgmexico, ...","[happy, new, yearpictwittercombhopdpqgmexico, ...","[happi, new, yearpictwittercombhopdpqgmexico, ..."
2,1/1/20,"Get this straightened out, Governor @ GavinNew...",Get this straightened out Governor GavinNewso...,"[get, this, straightened, out, governor, gavin...","[get, straightened, governor, gavinnewsomwonde...","[get, straighten, governor, gavinnewsomwond, a..."
3,1/10/18,"Today, it was my great honor to sign a new Exe...",Today it was my great honor to sign a new Exec...,"[today, it, was, my, great, honor, to, sign, a...","[today, great, honor, sign, new, executive, or...","[today, great, honor, sign, new, execut, order..."
4,1/10/19,The Mainstream Media has NEVER been more disho...,The Mainstream Media has NEVER been more disho...,"[the, mainstream, media, has, never, been, mor...","[mainstream, media, never, dishonest, nbc, msn...","[mainstream, media, never, dishonest, nbc, msn..."


In [30]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

trumpTweet_groupedDate['Tweet_lemmatized'] = trumpTweet_groupedDate['removed_stopwords'].apply(lambda x: lemmatizer(x))
trumpTweet_groupedDate.head()

,date,content,cleaned_content,Tweet_tokenized,removed_stopwords,stemmed_words,Tweet_lemmatized
0,1/1/18,HAPPY NEW YEAR! We are MAKING AMERICA GREAT AG...,HAPPY NEW YEAR We are MAKING AMERICA GREAT AGA...,"[happy, new, year, we, are, making, america, g...","[happy, new, year, making, america, great, muc...","[happi, new, year, make, america, great, much,...","[happy, new, year, making, america, great, muc..."
1,1/1/19,HAPPY NEW YEAR!pic.twitter.com/bHoPDPQ7G6-MEXI...,HAPPY NEW YEARpictwittercombHoPDPQGMEXICO IS P...,"[happy, new, yearpictwittercombhopdpqgmexico, ...","[happy, new, yearpictwittercombhopdpqgmexico, ...","[happi, new, yearpictwittercombhopdpqgmexico, ...","[happy, new, yearpictwittercombhopdpqgmexico, ..."
2,1/1/20,"Get this straightened out, Governor @ GavinNew...",Get this straightened out Governor GavinNewso...,"[get, this, straightened, out, governor, gavin...","[get, straightened, governor, gavinnewsomwonde...","[get, straighten, governor, gavinnewsomwond, a...","[get, straightened, governor, gavinnewsomwonde..."
3,1/10/18,"Today, it was my great honor to sign a new Exe...",Today it was my great honor to sign a new Exec...,"[today, it, was, my, great, honor, to, sign, a...","[today, great, honor, sign, new, executive, or...","[today, great, honor, sign, new, execut, order...","[today, great, honor, sign, new, executive, or..."
4,1/10/19,The Mainstream Media has NEVER been more disho...,The Mainstream Media has NEVER been more disho...,"[the, mainstream, media, has, never, been, mor...","[mainstream, media, never, dishonest, nbc, msn...","[mainstream, media, never, dishonest, nbc, msn...","[mainstream, medium, never, dishonest, nbc, ms..."


In [31]:
trumpTweet_groupedDate['Tweet_lemmatized_stemmed'] = trumpTweet_groupedDate['stemmed_words'].apply(lambda x: lemmatizer(x))

In [32]:
trumpTweet_groupedDate.head()

,date,content,cleaned_content,Tweet_tokenized,removed_stopwords,stemmed_words,Tweet_lemmatized,Tweet_lemmatized_stemmed
0,1/1/18,HAPPY NEW YEAR! We are MAKING AMERICA GREAT AG...,HAPPY NEW YEAR We are MAKING AMERICA GREAT AGA...,"[happy, new, year, we, are, making, america, g...","[happy, new, year, making, america, great, muc...","[happi, new, year, make, america, great, much,...","[happy, new, year, making, america, great, muc...","[happi, new, year, make, america, great, much,..."
1,1/1/19,HAPPY NEW YEAR!pic.twitter.com/bHoPDPQ7G6-MEXI...,HAPPY NEW YEARpictwittercombHoPDPQGMEXICO IS P...,"[happy, new, yearpictwittercombhopdpqgmexico, ...","[happy, new, yearpictwittercombhopdpqgmexico, ...","[happi, new, yearpictwittercombhopdpqgmexico, ...","[happy, new, yearpictwittercombhopdpqgmexico, ...","[happi, new, yearpictwittercombhopdpqgmexico, ..."
2,1/1/20,"Get this straightened out, Governor @ GavinNew...",Get this straightened out Governor GavinNewso...,"[get, this, straightened, out, governor, gavin...","[get, straightened, governor, gavinnewsomwonde...","[get, straighten, governor, gavinnewsomwond, a...","[get, straightened, governor, gavinnewsomwonde...","[get, straighten, governor, gavinnewsomwond, a..."
3,1/10/18,"Today, it was my great honor to sign a new Exe...",Today it was my great honor to sign a new Exec...,"[today, it, was, my, great, honor, to, sign, a...","[today, great, honor, sign, new, executive, or...","[today, great, honor, sign, new, execut, order...","[today, great, honor, sign, new, executive, or...","[today, great, honor, sign, new, execut, order..."
4,1/10/19,The Mainstream Media has NEVER been more disho...,The Mainstream Media has NEVER been more disho...,"[the, mainstream, media, has, never, been, mor...","[mainstream, media, never, dishonest, nbc, msn...","[mainstream, media, never, dishonest, nbc, msn...","[mainstream, medium, never, dishonest, nbc, ms...","[mainstream, medium, never, dishonest, nbc, ms..."


In [33]:
#Keeping all processed words in a dictionary which shall be later used for creating bag of words

dictionary = gensim.corpora.Dictionary(trumpTweet_groupedDate['Tweet_lemmatized_stemmed'])


In [34]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 administr
1 afghanistan
2 aid
3 along
4 america
5 anyon
6 billion
7 changewil
8 dc
9 deal
10 deceit

In [35]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

###Creating Bag of word (words and their frequencies)

In [37]:
bow_corpus = [dictionary.doc2bow(doc) for doc in trumpTweet_groupedDate['Tweet_lemmatized_stemmed']]
len(bow_corpus)

Out[157]: 1148

In [38]:
bow_doc_1147 = bow_corpus[1147]
for i in range(len(bow_doc_1147)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_1147[i][0], 
                                               dictionary[bow_doc_1147[i][0]], 
bow_doc_1147[i][1]))

Word 0 ("administr") appears 3 time.
Word 8 ("despit") appears 1 time.
Word 10 ("done") appears 4 time.
Word 11 ("everi") appears 1 time.
Word 12 ("fail") appears 2 time.
Word 17 ("give") appears 1 time.
Word 20 ("help") appears 4 time.
Word 25 ("last") appears 2 time.
Word 26 ("leader") appears 2 time.
Word 27 ("leav") appears 2 time.
Word 33 ("mani") appears 2 time.
Word 34 ("much") appears 5 time.
Word 36 ("noth") appears 6 time.
Word 37 ("obama") appears 1 time.
Word 41 ("state") appears 2 time.
Word 44 ("think") appears 1 time.
Word 45 ("thought") appears 1 time.
Word 46 ("time") appears 2 time.
Word 47 ("today") appears 1 time.
Word 48 ("u") appears 1 time.
Word 49 ("unit") appears 1 time.
Word 50 ("washington") appears 1 time.
Word 52 ("work") appears 1 time.
Word 53 ("year") appears 3 time.
Word 54 ("also") appears 2 time.
Word 55 ("alway") appears 2 time.
Word 56 ("away") appears 1 time.
Word 57 ("bad") appears 6 time.
Word 58 ("better") appears 1 time.
Word 59 ("big") appears 2 time.
Word 62 ("border") appears 1 time.
Word 65 ("case") appears 1 time.
Word 67 ("countri") appears 2 time.
Word 68 ("crime") appears 1 time.
Word 69 ("democrat") appears 2 time.
Word 74 ("everyon") appears 1 time.
Word 78 ("fake") appears 3 time.
Word 85 ("get") appears 3 time.
Word 86 ("go") appears 2 time.
Word 87 ("good") appears 2 time.
Word 88 ("got") appears 1 time.
Word 89 ("guy") appears 1 time.
Word 90 ("happen") appears 1 time.
Word 99 ("like") appears 3 time.
Word 101 ("lot") appears 1 time.
Word 102 ("low") appears 1 time.
Word 104 ("medium") appears 5 time.
Word 109 ("name") appears 1 time.
Word 111 ("never") appears 2 time.
Word 112 ("news") appears 4 time.
Word 118 ("presid") appears 3 time.
Word 121 ("problem") appears 1 time.
Word 123 ("real") appears 1 time.
Word 124 ("reelect") appears 1 time.
Word 129 ("see") appears 2 time.
Word 145 ("trump") appears 3 time.
Word 146 ("usa") appears 1 time.
Word 148 ("wall") appears 1 time.
Word 150 ("way") appears 3 time.
Word 152 ("would") appears 2 time.
Word 155 ("approv") appears 1 time.
Word 156 ("choic") appears 1 time.
Word 158 ("first") appears 1 time.
Word 160 ("governor") appears 2 time.
Word 165 ("other") appears 2 time.
Word 172 ("achiev") appears 1 time.
Word 178 ("american") appears 1 time.
Word 180 ("back") appears 1 time.
Word 188 ("court") appears 1 time.
Word 191 ("dem") appears 1 time.
Word 192 ("deserv") appears 1 time.
Word 200 ("found") appears 1 time.
Word 201 ("friend") appears 1 time.
Word 203 ("grate") appears 1 time.
Word 204 ("hero") appears 1 time.
Word 206 ("histori") appears 1 time.
Word 211 ("know") appears 2 time.
Word 214 ("look") appears 1 time.
Word 218 ("need") appears 2 time.
Word 228 ("republican") appears 4 time.
Word 230 ("run") appears 2 time.
Word 234 ("sign") appears 1 time.
Word 239 ("take") appears 2 time.
Word 242 ("thank") appears 2 time.
Word 248 ("win") appears 1 time.
Word 254 ("believ") appears 1 time.
Word 257 ("cnn") appears 3 time.
Word 259 ("dishonest") appears 1 time.
Word 261 ("even") appears 2 time.
Word 265 ("gone") appears 1 time.
Word 267 ("hous") appears 1 time.
Word 268 ("import") appears 2 time.
Word 271 ("left") appears 1 time.
Word 274 ("meet") appears 2 time.
Word 275 ("msnbc") appears 1 time.
Word 278 ("one") appears 3 time.
Word 280 ("parti") appears 2 time.
Word 281 ("polit") appears 1 time.
Word 286 ("record") appears 1 time.
Word 288 ("report") appears 2 time.
Word 293 ("say") appears 2 time.
Word 294 ("senat") appears 1 time.
Word 295 ("sound") appears 1 time.
Word 297 ("stori") appears 2 time.
Word 304 ("white") appears 1 time.
Word 305 ("wors") appears 1 time.
Word 309 ("anyth") appears 1 time.
Word 312 ("busi") appears 1 time.
Word 313 ("come") appears 3 time.
Word 315 ("corrupt") appears 1 time.
Word 320 ("economi") appears 2 time.
Word 321 ("elect") appears 1 time.
Word 322 ("enemi") appears 1 time.
Word 324 ("famili") appears 1 time.
Word 328 ("hit") appears 1 time.
Word 332 ("live") appears 1 time.
Word 33

In [39]:
%sh pip install wordcloud

Requirement already satisfied: wordcloud in /databricks/python3/lib/python3.7/site-packages (1.6.0)
Requirement already satisfied: matplotlib in /databricks/python3/lib/python3.7/site-packages (from wordcloud) (3.0.3)
Requirement already satisfied: numpy>=1.6.1 in /databricks/python3/lib/python3.7/site-packages (from wordcloud) (1.16.2)
Requirement already satisfied: pillow in /databricks/python3/lib/python3.7/site-packages (from wordcloud) (7.1.2)
Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib->wordcloud) (2.4.6)
Requirement already satisfied: cycler>=0.10 in /databricks/python3/lib/python3.7/site-packages (from matplotlib->wordcloud) (0.10.0)
Requirement already satisfied: python-dateutil>=2.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib->wordcloud) (2.8.0)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib->wordcloud) (1.1.0)
Requirement already satisfied: six in /databricks/python3/lib/python3.7/site-packages (from cycler>=0.10->matplotlib->wordcloud) (1.12.0)
Requirement already satisfied: setuptools in /usr/lib/python3.7/site-packages (from kiwisolver>=1.0.1->matplotlib->wordcloud) (40.8.0)

## Visualization of Words (Exploratory Data Analysis of Texts)

#### Visualizing the word variety in the tweet content

In [42]:

from wordcloud import WordCloud
stopwords = set(STOPWORDS)

def show_wordcloud(data):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=100,
        max_font_size=30,
        scale=3,
        random_state=1)
   
    wordcloud=wordcloud.generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(trumpTweet_groupedDate['content'])

##Vectorizing and Visualizing texts

####Importing packages for topic modeling

In [45]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.10338954721704342),
 (1, 0.20526360634724747),
 (2, 0.13905097002015268),
 (3, 0.06110978274103288),
 (4, 0.15451966125825656),
 (5, 0.10790771733177193),
 (6, 0.1748671812467633),
 (7, 0.07873475346510463),
 (8, 0.13731607823613615),
 (9, 0.09920996502166929),
 (10, 0.07216231980868927),
 (11, 0.1232924338499024),
 (12, 0.11781320185307531),
 (13, 0.13964146432293828),
 (14, 0.20722174378266284),
 (15, 0.21582353529372303),
 (16, 0.17831990246111082),
 (17, 0.10277295319867852),
 (18, 0.2958944552036824),
 (19, 0.14084153618582013),
 (20, 0.08922829193272985),
 (21, 0.1748671812467633),
 (22, 0.09532174246886013),
 (23, 0.14726254182287177),
 (24, 0.21819841246216523),
 (25, 0.0921918834332756),
 (26, 0.11903073068180933),
 (27, 0.12986486750631776),
 (28, 0.14084153618582013),
 (29, 0.1264819414116277),
 (30, 0.14145139133685106),
 (31, 0.08201812343902225),
 (32, 0.05427960640517397),
 (33, 0.050911808734296606),
 (34, 0.12546319194305075),
 (35, 0.12427275525018122),
 (36, 0.08014751730138457),
 (37, 0.10496151570023658),
 (38, 0.13674956381807338),
 (39, 0.09863764009796361),
 (40, 0.15150264592319324),
 (41, 0.042604757770272256),
 (42, 0.13453932674219296),
 (43, 0.19977053305284448),
 (44, 0.10723943160902381),
 (45, 0.12936885365466555),
 (46, 0.05201197737832045),
 (47, 0.06889413589851792),
 (48, 0.044299041725282255),
 (49, 0.0675733359904328),
 (50, 0.12509262838085838),
 (51, 0.21819841246216523),
 (52, 0.054926209131383),
 (53, 0.19278009503184187)]

####2) Visualizing Unigrams in the tweet contents before removing stop words

In [47]:
def get_top_ngram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) 
                  for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:10]
top_n_bigrams=get_top_ngram(trumpTweet_groupedDate['content'],1)[:10]
x,y=map(list,zip(*top_n_bigrams))
sns.barplot(x=y,y=x)

From the above Visualization we reckon that the frequency of the stopwords outcount the frequency of any potential significant word. Hence, defying the idea of direct proportionality between the number of words and its importance. In order to handle this situation we remove the stopwords and visualize the documents

####2) Visualizing Unigrams in the tweet contents after removing stop words

In [50]:
def get_top_ngram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(n, n), stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) 
                  for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:10]
top_n_bigrams=get_top_ngram(trumpTweet_groupedDate['content'],1)[:10]
x,y=map(list,zip(*top_n_bigrams))
sns.barplot(x=y,y=x)

####Approach 1 (Topic Model created using Bag of words i.e. Words and their frequencies)

In [52]:
#Topic Modeling algorithm

lda_model = gensim.models.LdaMulticore(bow_corpus, id2word=dictionary, passes=4, workers=2)

In [53]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.012*"prime" + 0.012*"minist" + 0.010*"japan" + 0.009*"mani" + 0.009*"back" + 0.009*"get" + 0.008*"one" + 0.008*"u" + 0.007*"border" + 0.007*"time"
Topic: 1 
Words: 0.019*"presid" + 0.012*"korea" + 0.011*"trump" + 0.010*"north" + 0.009*"u" + 0.009*"china" + 0.008*"much" + 0.008*"go" + 0.008*"look" + 0.008*"mani"
Topic: 2 
Words: 0.017*"school" + 0.015*"u" + 0.014*"gun" + 0.012*"nation" + 0.011*"get" + 0.010*"would" + 0.010*"news" + 0.010*"mani" + 0.009*"fake" + 0.009*"must"
Topic: 3 
Words: 0.012*"presid" + 0.012*"want" + 0.011*"countri" + 0.010*"state" + 0.009*"call" + 0.009*"republican" + 0.009*"democrat" + 0.008*"never" + 0.008*"tax" + 0.007*"impeach"
Topic: 4 
Words: 0.014*"democrat" + 0.013*"year" + 0.012*"countri" + 0.012*"nfl" + 0.011*"get" + 0.011*"total" + 0.011*"border" + 0.009*"vote" + 0.009*"american" + 0.008*"support"
Topic: 5 
Words: 0.014*"mani" + 0.011*"presid" + 0.010*"trump" + 0.008*"even" + 0.008*"record" + 0.008*"fake" + 0.007*"nation" + 0.007*"democrat" + 0.007*"never" + 0.007*"hous"
Topic: 6 
Words: 0.036*"countri" + 0.025*"charg" + 0.020*"bad" + 0.019*"ban" + 0.019*"judg" + 0.018*"certain" + 0.017*"come" + 0.016*"tariff" + 0.015*"coverag" + 0.015*"nytim"
Topic: 7 
Words: 0.017*"thank" + 0.017*"news" + 0.016*"michigan" + 0.013*"parti" + 0.013*"year" + 0.012*"schiff" + 0.012*"back" + 0.011*"talk" + 0.011*"show" + 0.011*"fake"
Topic: 8 
Words: 0.015*"countri" + 0.011*"state" + 0.011*"democrat" + 0.010*"unit" + 0.009*"want" + 0.009*"mani" + 0.009*"noth" + 0.008*"tax" + 0.008*"big" + 0.008*"border"
Topic: 9 
Words: 0.014*"like" + 0.013*"vote" + 0.012*"go" + 0.011*"ever" + 0.011*"north" + 0.011*"korea" + 0.011*"much" + 0.010*"car" + 0.010*"nation" + 0.010*"russia"
Topic: 10 
Words: 0.026*"report" + 0.018*"fbi" + 0.016*"clinton" + 0.015*"comey" + 0.011*"mani" + 0.010*"good" + 0.010*"unit" + 0.010*"total" + 0.009*"news" + 0.009*"year"
Topic: 11 
Words: 0.014*"job" + 0.012*"american" + 0.010*"republican" + 0.010*"want" + 0.010*"vote" + 0.010*"get" + 0.010*"news" + 0.009*"win" + 0.009*"year" + 0.008*"time"
Topic: 12 
Words: 0.025*"iraq" + 0.021*"democrat" + 0.016*"want" + 0.015*"report" + 0.013*"trial" + 0.012*"new" + 0.012*"mani" + 0.012*"noth" + 0.011*"u" + 0.011*"go"
Topic: 13 
Words: 0.020*"would" + 0.020*"get" + 0.019*"medium" + 0.016*"arizona" + 0.015*"fake" + 0.015*"u" + 0.015*"hard" + 0.011*"courag" + 0.011*"countri" + 0.010*"news"
Topic: 14 
Words: 0.014*"news" + 0.013*"fake" + 0.012*"trump" + 0.010*"meet" + 0.010*"time" + 0.009*"presid" + 0.009*"north" + 0.008*"even" + 0.008*"want" + 0.008*"poll"
Topic: 15 
Words: 0.016*"vote" + 0.015*"countri" + 0.012*"job" + 0.012*"get" + 0.010*"state" + 0.009*"border" + 0.009*"u" + 0.008*"new" + 0.008*"total" + 0.007*"democrat"
Topic: 16 
Words: 0.025*"border" + 0.019*"democrat" + 0.014*"countri" + 0.012*"want" + 0.012*"law" + 0.011*"trump" + 0.010*"stop" + 0.010*"total" + 0.009*"u" + 0.009*"must"
Topic: 17 
Words: 0.014*"state" + 0.013*"countri" + 0.012*"presid" + 0.012*"trump" + 0.011*"democrat" + 0.010*"u" + 0.010*"want" + 0.009*"american" + 0.009*"israel" + 0.008*"unit"
Topic: 18 
Words: 0.019*"rico" + 0.019*"puerto" + 0.015*"work" + 0.013*"new" + 0.012*"border" + 0.011*"american" + 0.010*"get" + 0.009*"trump" + 0.009*"year" + 0.008*"wall"
Topic: 19 
Words: 0.012*"border" + 0.012*"mani" + 0.012*"democrat" + 0.011*"want" + 0.010*"countri" + 0.010*"wall" + 0.009*"fight" + 0.008*"go" + 0.008*"make" + 0.008*"new"
Topic: 20 
Words: 0.021*"presid" + 0.021*"elect" + 0.016*"get" + 0.012*"trump" + 0.012*"win" + 0.012*"republican" + 0.012*"host" + 0.009*"time" + 0.008*"period" + 0.008*"medium"
Topic: 21 
Words: 0.024*"come" + 0.012*"must" + 0.010*"today" + 0.010*"make" + 0.009*"june" + 0.009*"year" + 0.008*"countri" + 0.008*"illeg" + 0.008*"news" + 0.008*"u"
Topic: 22 
Words: 0.018*"thank" + 0.012*"book" + 0.012*"must" + 0.011*"time" + 0.010*"america" + 0.009*"state" + 0.009*"commit" + 0.008*"let" + 0.008*"mani" + 0.008*"job"
Topic: 23 
Words: 0.015*"borde

In [54]:
print('The nltk version is {}.'.format(nltk.__version__))

The nltk version is 3.5.

In [55]:
#One compact function for cleaning/preprocessing text for using in the vectorizer
def clean_text(text):
    text_tc = ''.join([w if ord(w) < 128 else ' ' for w in text])
    text_lc = "".join([word.lower() for word in text_tc if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    #text_nc = re.sub(r'[^\x00-\x7F]+','', text_rc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

<>:7: DeprecationWarning: invalid escape sequence \W
<>:7: DeprecationWarning: invalid escape sequence \W
<command-392745532674054>:7: DeprecationWarning: invalid escape sequence \W
 tokens = re.split('\W+', text_rc) # tokenization

####Approach 2 (Topic model created using vecorized words)

**Note**: The disadvantage of the previous approach is that the model is trained upon only the words that it finds within the document and based on their frequencies. On the other hand the advantage of the subsequent model is that it creates a word vector dataframe using various possibilities of words (n-grams). This helps the machine learn on the words that are present and the words that could have been there (as absent words)

In [58]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [59]:
#Vectorizing the words
countVectorizer = CountVectorizer(analyzer=clean_text, token_pattern='[a-zA-Z0-9]{3,}', lowercase=True) 
countVector = countVectorizer.fit_transform(trumpTweet_groupedDate['content'])
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))

1148 Number of tweets has 16133 words

In [60]:
countVector

Out[170]: <1148x16133 sparse matrix of type '<class 'numpy.int64'>'
	with 129289 stored elements in Compressed Sparse Row format>

In [61]:
type(corpus_tfidf)

Out[171]: gensim.interfaces.TransformedCorpus

### Word Vector

In [63]:
count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
count_vect_df

,,aaa,aap,aapour,ab,abandon,abba,abbaspictwittercomuzpivvrpgreat,abbott,abc,abcmr,abcnew,abcpoliticspictwittercomjixcsnzcpictwittercomyowcqlmad,abcwashington,abcworldnew,abduct,abdul,abdullah,abe,abedin,abepictwittercomeyyfwdqtoday,abepictwittercomlragojvdbplay,abeshinzo,abeshinzopictwittercomnwwxjlkxhthank,abeshinzopictwittercomqxmpcgrycfpastor,abeshinzopictwittercomuwejqnbexepictwittercomgraqvxrsmitch,abethank,abid,abig,abil,abl,ablepictwittercommctybguepi,aboard,abolish,abort,abound,aboutdepart,aboutth,aboutw,abraham,...,youtub,youv,youw,youwhat,youwhen,youwher,youwheth,youwhi,youwhil,youwho,youwil,yovanovitch,yrs,yrsdo,yucca,yulin,yuma,z,zaino,zapotoski,zealand,zealandpictwittercomtegqfkcmsjust,zeldin,zelenski,zell,zero,zfor,zika,zink,zito,zogbi,zoldan,zone,zoo,zoomappl,zoomsen,zte,zucker,zuckerberg,zuker
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [64]:
%sh pip install spacy

Requirement already satisfied: spacy in /databricks/python3/lib/python3.7/site-packages (2.2.4)
Requirement already satisfied: requests<3.0.0,>=2.13.0 in /databricks/python3/lib/python3.7/site-packages (from spacy) (2.21.0)
Requirement already satisfied: cymem<2.1.0,>=2.0.2 in /databricks/python3/lib/python3.7/site-packages (from spacy) (2.0.3)
Requirement already satisfied: plac<1.2.0,>=0.9.6 in /databricks/python3/lib/python3.7/site-packages (from spacy) (1.1.3)
Requirement already satisfied: preshed<3.1.0,>=3.0.2 in /databricks/python3/lib/python3.7/site-packages (from spacy) (3.0.2)
Requirement already satisfied: tqdm<5.0.0,>=4.38.0 in /databricks/python3/lib/python3.7/site-packages (from spacy) (4.45.0)
Requirement already satisfied: murmurhash<1.1.0,>=0.28.0 in /databricks/python3/lib/python3.7/site-packages (from spacy) (1.0.2)
Requirement already satisfied: thinc==7.4.0 in /databricks/python3/lib/python3.7/site-packages (from spacy) (7.4.0)
Requirement already satisfied: numpy>=1.15.0 in /databricks/python3/lib/python3.7/site-packages (from spacy) (1.16.2)
Requirement already satisfied: catalogue<1.1.0,>=0.0.7 in /databricks/python3/lib/python3.7/site-packages (from spacy) (1.0.0)
Requirement already satisfied: srsly<1.1.0,>=1.0.2 in /databricks/python3/lib/python3.7/site-packages (from spacy) (1.0.2)
Requirement already satisfied: blis<0.5.0,>=0.4.0 in /databricks/python3/lib/python3.7/site-packages (from spacy) (0.4.1)
Requirement already satisfied: wasabi<1.1.0,>=0.4.0 in /databricks/python3/lib/python3.7/site-packages (from spacy) (0.6.0)
Requirement already satisfied: setuptools in /usr/lib/python3.7/site-packages (from spacy) (40.8.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests<3.0.0,>=2.13.0->spacy) (2019.3.9)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests<3.0.0,>=2.13.0->spacy) (2.8)
Requirement already satisfied: urllib3<1.25,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests<3.0.0,>=2.13.0->spacy) (1.24.1)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /databricks/python3/lib/python3.7/site-packages (from requests<3.0.0,>=2.13.0->spacy) (3.0.4)
Requirement already satisfied: importlib-metadata>=0.20; python_version < "3.8" in /databricks/python3/lib/python3.7/site-packages (from catalogue<1.1.0,>=0.0.7->spacy) (1.6.0)
Requirement already satisfied: zipp>=0.5 in /databricks/python3/lib/python3.7/site-packages (from importlib-metadata>=0.20; python_version < "3.8"->catalogue<1.1.0,>=0.0.7->spacy) (3.1.0)

##Creating LDA model for topic modeling

In [66]:
import spacy as sp


In [67]:
# Materialize the sparse data
data_dense = countVector.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity: 0.6980789694376298 %

In [68]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD

In [69]:
lda_model = LatentDirichletAllocation(n_components = 20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(countVector)

In [70]:
print(lda_model)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
 evaluate_every=-1, learning_decay=0.7,
 learning_method='online', learning_offset=10.0,
 max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
 n_components=20, n_jobs=-1, n_topics=None, perp_tol=0.1,
 random_state=100, topic_word_prior=None,
 total_samples=1000000.0, verbose=0)

In [71]:
# Log Likelyhood: Higher the better (Likelyhood score tells us how good the model is performing. The more the magnitude the better the model)
print("Log Likelihood: ", lda_model.score(countVector))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(countVector))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood: -1276599.725280423
Perplexity: 2737.463145699325
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'n_topics': None,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}

In [72]:
from sklearn.model_selection import GridSearchCV

####Running the Grid Search on LDA to find the best model

In [74]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(countVector)

/databricks/python/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
 warnings.warn(CV_WARNING, FutureWarning)
Out[181]: GridSearchCV(cv='warn', error_score='raise-deprecating',
 estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
 evaluate_every=-1, learning_decay=0.7,
 learning_method='batch', learning_offset=10.0,
 max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
 n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
 random_state=None, topic_word_prior=None,
 total_samples=1000000.0, verbose=0),
 fit_params=None, iid='warn', n_jobs=None,
 param_grid={'n_components': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
 pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
 scoring=None, verbose=0)

In [75]:
# Best Model Parameters
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(countVector))

Best Model's Params: {'learning_decay': 0.7, 'n_components': 10}
Best Log Likelihood Score: -506056.5870967257
Model Perplexity: 2787.436974171556

In [76]:
%sh pip install pyLDAvis

Requirement already satisfied: pyLDAvis in /databricks/python3/lib/python3.7/site-packages (2.1.2)
Requirement already satisfied: funcy in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (1.14)
Requirement already satisfied: numexpr in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (2.7.1)
Requirement already satisfied: pytest in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (5.4.1)
Requirement already satisfied: jinja2>=2.7.2 in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (2.11.2)
Requirement already satisfied: wheel>=0.23.0 in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (0.33.1)
Requirement already satisfied: future in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (0.18.2)
Requirement already satisfied: scipy>=0.18.0 in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (1.2.1)
Requirement already satisfied: numpy>=1.9.2 in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (1.16.2)
Requirement already satisfied: pandas>=0.17.0 in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (0.24.2)
Requirement already satisfied: joblib>=0.8.4 in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (0.14.1)
Requirement already satisfied: pluggy<1.0,>=0.12 in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (0.13.1)
Requirement already satisfied: py>=1.5.0 in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (1.8.1)
Requirement already satisfied: wcwidth in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (0.1.7)
Requirement already satisfied: importlib-metadata>=0.12; python_version < "3.8" in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (1.6.0)
Requirement already satisfied: attrs>=17.4.0 in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (19.3.0)
Requirement already satisfied: packaging in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (20.3)
Requirement already satisfied: more-itertools>=4.0.0 in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (8.2.0)
Requirement already satisfied: MarkupSafe>=0.23 in /databricks/python3/lib/python3.7/site-packages (from jinja2>=2.7.2->pyLDAvis) (1.1.1)
Requirement already satisfied: python-dateutil>=2.5.0 in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.17.0->pyLDAvis) (2.8.0)
Requirement already satisfied: pytz>=2011k in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.17.0->pyLDAvis) (2018.9)
Requirement already satisfied: zipp>=0.5 in /databricks/python3/lib/python3.7/site-packages (from importlib-metadata>=0.12; python_version < "3.8"->pytest->pyLDAvis) (3.1.0)
Requirement already satisfied: six in /databricks/python3/lib/python3.7/site-packages (from packaging->pytest->pyLDAvis) (1.12.0)
Requirement already satisfied: pyparsing>=2.0.2 in /databricks/python3/lib/python3.7/site-packages (from packaging->pytest->pyLDAvis) (2.4.6)

In [77]:
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [78]:
model.cv_results_

Out[184]: /databricks/python/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
 warnings.warn(*warn_args, **warn_kwargs)
/databricks/python/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
 warnings.warn(*warn_args, **warn_kwargs)
/databricks/python/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
 warnings.warn(*warn_args, **warn_kwargs)
/databricks/python/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
 warnings.warn(*warn_args, **warn_kwargs)
/databricks/python/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('std_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
 warnings.warn(*warn_args, **warn_kwargs)
{'mean_fit_time': array([6.27047539, 6.5446771 , 6.50771721, 7.09274689, 8.50355148,
 6.27545722, 6.17467602, 6.26377002, 7.45716882, 8.27788059,
 6.23154179, 6.19051814, 6.49158374, 7.06748382, 8.24183035]),
 'std_fit_time': array([0.09507053, 0.25396342, 0.10191974, 0.48230666, 0.58930248,
 0.17651375, 0.23413818, 0.15455335, 0.66874048, 0.62443566,
 0.31350969, 0.14169094, 0.20437963, 0.50867035, 0.65881781]),
 'mean_score_time': array([0.39176687, 0.4184703 , 0.46927571, 0.52991438, 0.55580147,
 0.37833587, 0.44918633, 0.4763337 , 0.48679964, 0.56893953,
 0.36798159, 0.43505526, 0.43284432, 0.48163064, 0.56595198]),
 'std_score_time': array([0.0382165 , 0.03081849, 0.02886443, 0.07510833, 0.05923667,
 0.02292153, 0.10722147, 0.01821744, 0.05381171, 0.09675029,
 0.00349923, 0.01795223, 0.01290628, 0.04786581, 0.05646896]),
 'param_learning_decay': masked_array(data=[0.5, 0.5, 0.5, 0.5, 0.5, 0.7, 0.7, 0.7, 0.7, 0.7, 0.9,
 0.9, 0.9, 0.9, 0.9],
 mask=[False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False],
 fill_value='?',
 dtype=object),
 'param_n_components': masked_array(data=[10, 15, 20, 25, 30, 10, 15, 20, 25, 30, 10, 15, 20, 25,
 30],
 mask=[False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False],
 fill_value='?',
 dtype=object),
 'params': [{'learning_decay': 0.5, 'n_components': 10},
 {'learning_decay': 0.5, 'n_components': 15},
 {'learning_decay': 0.5, 'n_components': 20},
 {'learning_decay': 0.5, 'n_components': 25},
 {'learning_decay': 0.5, 'n_components': 30},
 {'learning_decay': 0.7, 'n_components': 10},
 {'learning_decay': 0.7, 'n_components': 15},
 {'learning_decay': 0.7, 'n_components': 20},
 {'learning_decay': 0.7, 'n_components': 25},
 {'learning_decay': 0.7, 'n_components': 30},
 {'learning_decay': 0.9, 'n_components': 10},
 {'learning_decay': 0.9, 'n_components': 15},
 {'learning_decay': 0.9, 'n_components': 20},
 {'learning_decay': 0.9, 'n_components': 25},
 {'learning_decay': 0.9, 'n_components': 30}],
 'split0_test_score': array([-543990.18386786, -574402.87858554, -621395.03987572,
 -650872.12299184, -687888.98830507, -538980.13950121,
 -570259.36859905, -614339.80726121, -649165.1612577 ,
 -689574.30602188, -543454.11772667, -584107.17610941,
 -610010.25283202, -653879.48222701, -689903.90995071]),
 'split1_test_score': array([-440997.6419

In [79]:
%sh pip install pyLDAvis

Requirement already satisfied: pyLDAvis in /databricks/python3/lib/python3.7/site-packages (2.1.2)
Requirement already satisfied: funcy in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (1.14)
Requirement already satisfied: numexpr in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (2.7.1)
Requirement already satisfied: pytest in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (5.4.1)
Requirement already satisfied: jinja2>=2.7.2 in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (2.11.2)
Requirement already satisfied: wheel>=0.23.0 in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (0.33.1)
Requirement already satisfied: future in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (0.18.2)
Requirement already satisfied: scipy>=0.18.0 in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (1.2.1)
Requirement already satisfied: numpy>=1.9.2 in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (1.16.2)
Requirement already satisfied: pandas>=0.17.0 in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (0.24.2)
Requirement already satisfied: joblib>=0.8.4 in /databricks/python3/lib/python3.7/site-packages (from pyLDAvis) (0.14.1)
Requirement already satisfied: py>=1.5.0 in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (1.8.1)
Requirement already satisfied: pluggy<1.0,>=0.12 in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (0.13.1)
Requirement already satisfied: wcwidth in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (0.1.7)
Requirement already satisfied: importlib-metadata>=0.12; python_version < "3.8" in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (1.6.0)
Requirement already satisfied: attrs>=17.4.0 in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (19.3.0)
Requirement already satisfied: packaging in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (20.3)
Requirement already satisfied: more-itertools>=4.0.0 in /databricks/python3/lib/python3.7/site-packages (from pytest->pyLDAvis) (8.2.0)
Requirement already satisfied: MarkupSafe>=0.23 in /databricks/python3/lib/python3.7/site-packages (from jinja2>=2.7.2->pyLDAvis) (1.1.1)
Requirement already satisfied: python-dateutil>=2.5.0 in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.17.0->pyLDAvis) (2.8.0)
Requirement already satisfied: pytz>=2011k in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.17.0->pyLDAvis) (2018.9)
Requirement already satisfied: zipp>=0.5 in /databricks/python3/lib/python3.7/site-packages (from importlib-metadata>=0.12; python_version < "3.8"->pytest->pyLDAvis) (3.1.0)
Requirement already satisfied: six in /databricks/python3/lib/python3.7/site-packages (from packaging->pytest->pyLDAvis) (1.12.0)
Requirement already satisfied: pyparsing>=2.0.2 in /databricks/python3/lib/python3.7/site-packages (from packaging->pytest->pyLDAvis) (2.4.6)

In [80]:
# Get Log Likelyhoods from Grid Search Output
n_components = [10, 15, 20, 25, 30]
log_likelyhoods_5 = [(model.cv_results_['mean_test_score'][index]) for index, gscore in enumerate(model.cv_results_['params']) if gscore['learning_decay']==0.5]
log_likelyhoods_7 = [(model.cv_results_['mean_test_score'][index]) for index, gscore in enumerate(model.cv_results_['params']) if gscore['learning_decay']==0.7]
log_likelyhoods_9 = [(model.cv_results_['mean_test_score'][index]) for index, gscore in enumerate(model.cv_results_['params']) if gscore['learning_decay']==0.9]
# log_likelyhoods_5 = [round(gscore.mean_score_time) for gscore in model.cv_results_['params'] if gscore['learning_decay']==0.5]
# log_likelyhoods_7 = [round(gscore.mean_validation_score) for gscore in model.cv_results_ if gscore.parameters['learning_decay']==0.7]
# log_likelyhoods_9 = [round(gscore.mean_validation_score) for gscore in model.cv_results_ if gscore.parameters['learning_decay']==0.9]


In [81]:
comp_log_5 = {'n_components': n_components, 'log_likelyhoods_5': log_likelyhoods_5}

###Comparing models visually

In [83]:
# Show graph
plt.figure(figsize=(12, 8))
plt.plot(n_components, log_likelyhoods_5, label='0.5')
plt.plot(n_components, log_likelyhoods_7, label='0.7')
plt.plot(n_components, log_likelyhoods_9, label='0.9')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

In [84]:
#Dominant topic in each document

# Create Document - Topic Matrix
lda_output = best_lda_model.transform(countVector)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(bow_corpus))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
#df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,9
1,0.29,0.00,0.00,0.00,0.00,0.00,0.00,0.16,0.00,0.55,9
2,0.61,0.00,0.00,0.00,0.00,0.38,0.00,0.00,0.00,0.00,0
3,0.00,0.00,0.73,0.00,0.00,0.00,0.00,0.00,0.00,0.26,2
4,0.00,0.00,0.00,0.00,0.13,0.00,0.00,0.00,0.00,0.86,9
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,0.00,8
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.22,0.00,0.77,9
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,9
8,0.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,9


In [85]:
#Review topics distribution across documents

df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,9,484
1,0,256
2,2,148
3,1,120
4,8,101
5,7,15
6,6,15
7,4,4
8,5,3
9,3,2


###Visualizing word vectors through PCAs

In [87]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, countVector, countVectorizer, mds='tsne')
panel

**Note : ** The vectors created from the words are clustered and plotted on the graph. Each circle represents a cluster of words under one topic. Distance between each circle is measured between centers. We made an interactive visualization that represents the list of words on the right in each topic (i.e. circle) on the left.

In [89]:
#Topics keywords
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = countVectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords

,,aaa,aap,aapour,ab,abandon,abba,abbaspictwittercomuzpivvrpgreat,abbott,abc,abcmr,abcnew,abcpoliticspictwittercomjixcsnzcpictwittercomyowcqlmad,abcwashington,abcworldnew,abduct,abdul,abdullah,abe,abedin,abepictwittercomeyyfwdqtoday,abepictwittercomlragojvdbplay,abeshinzo,abeshinzopictwittercomnwwxjlkxhthank,abeshinzopictwittercomqxmpcgrycfpastor,abeshinzopictwittercomuwejqnbexepictwittercomgraqvxrsmitch,abethank,abid,abig,abil,abl,ablepictwittercommctybguepi,aboard,abolish,abort,abound,aboutdepart,aboutth,aboutw,abraham,...,youtub,youv,youw,youwhat,youwhen,youwher,youwheth,youwhi,youwhil,youwho,youwil,yovanovitch,yrs,yrsdo,yucca,yulin,yuma,z,zaino,zapotoski,zealand,zealandpictwittercomtegqfkcmsjust,zeldin,zelenski,zell,zero,zfor,zika,zink,zito,zogbi,zoldan,zone,zoo,zoomappl,zoomsen,zte,zucker,zuckerberg,zuker
Topic0,76.597976,0.100000,2.798184,0.1,0.100011,2.794244,0.1,0.1,0.1,3.545452,0.100009,1.1,0.100005,0.1000,0.100002,0.100000,0.1,0.1,5.224402,0.100008,0.100000,0.1,0.100007,0.1,0.100000,1.099979,0.100003,0.100016,0.10001,1.100000,14.286271,0.100000,0.100005,0.100013,0.100046,0.100000,0.1,3.1,1.099974,1.10000,...,0.100000,0.100000,1.099999,1.1,0.100022,1.1,1.099994,0.100026,0.100010,0.100000,1.099997,2.099998,2.099989,0.1,0.100004,0.1,1.148269,0.100009,1.1,0.100015,0.100000,0.100000,2.010159,8.099986,1.1,37.437872,0.1,1.1,0.1,0.100000,0.100000,0.1,2.744538,1.099929,0.100002,1.1,0.100000,0.100013,1.1,0.100000
Topic1,34.426335,1.099998,0.100000,1.1,0.100000,0.100009,1.1,1.1,0.1,8.951250,0.100000,0.1,0.100000,0.7285,0.100009,1.099982,0.1,1.1,3.665837,0.651845,0.100000,0.1,8.085204,1.1,0.100000,0.100000,0.100000,0.100000,0.10000,0.100478,8.819019,0.100000,3.099955,0.100000,2.098198,0.100000,0.1,0.1,0.100000,1.09997,...,1.099985,0.100008,0.100000,0.1,0.100000,0.1,0.100000,0.100000,1.050687,0.100000,0.100000,0.100000,1.100000,0.1,0.100000,0.1,0.100005,0.100000,0.1,0.100000,0.100000,0.100000,0.100000,0.100000,0.1,3.172772,0.1,0.1,0.1,0.100000,0.100000,0.1,0.100011,0.100071,0.100000,0.1,0.100011,0.100000,0.1,1.099985
Topic2,24.118820,0.100000,1.401815,0.1,0.100000,1.894418,0.1,0.1,0.1,0.100009,0.100000,0.1,0.100000,0.1000,0.100000,0.100000,1.1,0.1,9.743061,0.100000,0.100036,1.1,1.114795,0.1,0.100000,0.100000,0.100023,0.100000,0.10000,0.100018,1.912617,1.099972,0.100011,2.829119,0.792412,0.100015,0.1,0.1,0.100000,0.10000,...,0.100000,1.099985,0.100018,0.1,1.099978,0.1,0.100000,0.100006,0.100000,1.099995,0.100000,0.100000,0.100027,0.1,0.100000,1.1,1.099980,0.100000,0.1,0.100000,3.584816,1.099982,0.100000,0.100000,0.1,6.859825,0.1,0.1,0.1,0.100000,0.100039,0.1,0.100009,0.100000,0.100000,0.1,0.100040,0.100000,0.1,0.100000
Topic3,0.100014,0.100000,0.100000,0.1,0.100000,0.100000,0.1,0.1,0.1,0.100000,0.100000,0.1,0.100000,0.1000,0.100000,0.100000,0.1,0.1,0.100000,0.100000,0.100000,0.1,0.100000,0.1,0.100000,0.100000,0.100000,0.100000,0.10000,0.100000,0.100007,0.100000,1.099991,0.100000,0.100000,0.100000,0.1,0.1,0.100000,0.10000,...,0.100000,0.100000,0.100000,0.1,0.100000,0.1,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.1,0.100000,0.1,1.051728,0.100000,0.1,0.100000,0.100000,0.100000,0.100000,0.100000,0.1,0.100000,0.1,0.1,0.1,0.100000,0.100000,0.1,0.100031,0.100000,0.100000,0.1,0.100000,0.100000,0.1,0.100000
Topic4,1.854665,0.100000,0.100000,0.1,0.100000,0.100000,0.1,0.1,0.1,0.100004,0.100000,0.1,1.099979,0.1000,0.100000,0.100000,0.1,0.1,0.100000,0.100000,0.100000,0.1,0.100000,0.1,0.100000,0.100000,0.100000,0.100000,0.10000,0.100000,0.100010,0.100000,0.100000,0.100000,0.101781,0.100000,0.1,0.1,0.100000,0.10000,...,0.100000,0.100000,1.099986,0.1,0.100000,0.1,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.1,0.100000,0.1,0.100000,0.100000,0.1,0.100000,0.100000,0.100000,0.100000,0.100012,0.1,0.100005,0.1,0.1,0.1,1.099991,0.100000,0.1,0.100000,0.100000,1.099988,0.1,0.100050,0.100000,0.1,0.100000
Topic5,1.160717,0.100000,0.100000,0.1,0.100000,0.100000,0.1,0.1,0.1,0.100000,0.100000,0.1,0.100000,0.1000,0.100000

In [90]:
#Getting top 15 keywords

# Show top n keywords for each topic
def show_topics(vectorizer= countVectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=countVectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,great,democrat,presid,impeach,peopl,republican,noth,get,vote,countri,us,state,go,border,thank
Topic 1,great,us,presid,china,state,year,trade,get,mani,work,countri,go,make,american,honor
Topic 2,great,peopl,state,countri,job,big,america,thank,border,work,new,make,american,get,love
Topic 3,elijah,cum,baltimor,corker,convinc,million,fort,myer,hurricaneharvey,contribut,inspir,racist,tennesse,space,marsha
Topic 4,switzerland,davo,rememb,harbor,pearl,daytona,enjoy,meddl,america,wisconsin,dnc,interview,econom,collect,foxandfriend
Topic 5,first,epa,honor,w,bless,nick,enforc,l,enter,law,polic,california,god,antitrump,jurisdict
Topic 6,american,thank,great,honor,commit,night,see,,vote,florida,berni,state,everyon,forward,obamacar
Topic 7,great,tax,go,must,korea,countri,peopl,cut,today,bad,north,nation,news,test,everi
Topic 8,great,news,fake,get,us,state,job,time,never,american,border,presid,peopl,work,republican
Topic 9,great,peopl,democrat,fake,news,presid,countri,trump,year,time,want,get,us,border,big


In [92]:
#Predicting topic of a new piece of text (i.e. a new tweet made by Donald Trump)

#Tokenization
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

#data_words = list(sent_to_words(data))

#print(data_words[:1])



In [93]:
%sh python3 -m spacy download en

Requirement already satisfied: en_core_web_sm==2.2.5 from https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.5/en_core_web_sm-2.2.5.tar.gz#egg=en_core_web_sm==2.2.5 in /databricks/python3/lib/python3.7/site-packages (2.2.5)
Requirement already satisfied: spacy>=2.2.2 in /databricks/python3/lib/python3.7/site-packages (from en_core_web_sm==2.2.5) (2.2.4)
Requirement already satisfied: requests<3.0.0,>=2.13.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (2.21.0)
Requirement already satisfied: cymem<2.1.0,>=2.0.2 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (2.0.3)
Requirement already satisfied: plac<1.2.0,>=0.9.6 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (1.1.3)
Requirement already satisfied: preshed<3.1.0,>=3.0.2 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (3.0.2)
Requirement already satisfied: tqdm<5.0.0,>=4.38.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (4.45.0)
Requirement already satisfied: murmurhash<1.1.0,>=0.28.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (1.0.2)
Requirement already satisfied: thinc==7.4.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (7.4.0)
Requirement already satisfied: numpy>=1.15.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (1.16.2)
Requirement already satisfied: catalogue<1.1.0,>=0.0.7 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (1.0.0)
Requirement already satisfied: srsly<1.1.0,>=1.0.2 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (1.0.2)
Requirement already satisfied: blis<0.5.0,>=0.4.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (0.4.1)
Requirement already satisfied: wasabi<1.1.0,>=0.4.0 in /databricks/python3/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (0.6.0)
Requirement already satisfied: setuptools in /usr/lib/python3.7/site-packages (from spacy>=2.2.2->en_core_web_sm==2.2.5) (40.8.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests<3.0.0,>=2.13.0->spacy>=2.2.2->en_core_web_sm==2.2.5) (2019.3.9)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests<3.0.0,>=2.13.0->spacy>=2.2.2->en_core_web_sm==2.2.5) (2.8)
Requirement already satisfied: urllib3<1.25,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests<3.0.0,>=2.13.0->spacy>=2.2.2->en_core_web_sm==2.2.5) (1.24.1)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /databricks/python3/lib/python3.7/site-packages (from requests<3.0.0,>=2.13.0->spacy>=2.2.2->en_core_web_sm==2.2.5) (3.0.4)
Requirement already satisfied: importlib-metadata>=0.20; python_version < "3.8" in /databricks/python3/lib/python3.7/site-packages (from catalogue<1.1.0,>=0.0.7->spacy>=2.2.2->en_core_web_sm==2.2.5) (1.6.0)
Requirement already satisfied: zipp>=0.5 in /databricks/python3/lib/python3.7/site-packages (from importlib-metadata>=0.20; python_version < "3.8"->catalogue<1.1.0,>=0.0.7->spacy>=2.2.2->en_core_web_sm==2.2.5) (3.1.0)
 ✔ Download and installation successful 
You can now load the model via spacy.load('en_core_web_sm')
 ✔ Linking successful 
/databricks/python3/lib/python3.7/site-packages/en_core_web_sm -->
/databricks/python3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')

In [94]:
import spacy

In [95]:
#Lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
#data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[:2])

###Implementation of the Topic Model

In [97]:
# Define function to predict topic for a given text document.
nlp = spacy.load('en', disable=['parser', 'ner'])

def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization

    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))

    # Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Step 3: Vectorize transform
    mytext_4 = countVectorizer.transform(mytext_3)

    # Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

# Predict the topic
mytext = ["I have instructed the United States Navy to shoot down and destroy any and all Iranian gunboats if they harass our ships at sea."]
topic, prob_scores = predict_topic(text = mytext)
print(prob_scores)

[[0.01250419 0.01250404 0.01250588 0.0125 0.0125 0.01250083
 0.0125024 0.0125011 0.01250184 0.88747971]]

##Part 2: Getting stock prices of National Grid

In [99]:
national_grid_stock_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true').option("delimiter", ",").option("escape", "\"").load('/FileStore/tables/NGG.csv')

In [100]:
national_grid_stock_df.show()

+----------+---------+---------+---------+---------+---------+-------+
 Date| Open| High| Low| Close|Adj Close| Volume|
+----------+---------+---------+---------+---------+---------+-------+
2017-01-23|63.810043|64.159386|63.722706|64.104805|54.674591| 459000|
2017-01-24|63.799126|64.181221|63.777294|63.897381|54.497673| 652000|
2017-01-25|64.104805|64.126640|63.635372|63.755459|54.376633| 425300|
2017-01-26|63.133186|63.242359|62.783844|63.176857|53.883141|1828600|
2017-01-27|63.329693|63.427948|63.122272|63.187775|53.892460| 427400|
2017-01-30|62.838428|63.133186|62.783844|62.893013|53.641052| 666600|
2017-01-31|63.351528|63.962883|63.286026|63.831879|54.441807| 845500|
2017-02-01|63.373363|63.722706|62.936680|63.351528|54.032120| 795200|
2017-02-02|63.198689|63.449783|63.111355|63.351528|54.032120| 479700|
2017-02-03|63.406113|63.613537|63.176857|63.329693|54.013500| 401800|
2017-02-06|63.569870|63.668121|63.384281|63.471615|54.134544| 551700|
2017-02-07|63.679039|64.497818|63.526199|64.323143|54.860798| 677200|
2017-02-08|65.447601|65.862442|65.131004|65.338425|55.726730| 894700|
2017-02-09|65.338425|66.331879|65.196510|65.360260|55.745354| 894900|
2017-02-10|65.032753|65.458519|65.032753|65.371178|55.754665| 371200|
2017-02-13|65.393013|65.447601|65.120087|65.382095|55.763981| 538600|
2017-02-14|65.633186|65.655022|64.901749|65.229256|55.633621| 474400|
2017-02-15|64.421394|65.065506|64.355896|65.065506|55.493965| 473000|
2017-02-16|64.956329|65.469429|64.956329|65.436684|55.810535| 311900|
2017-02-17|65.502182|65.556770|65.032753|65.382095|55.763981| 398600|
+----------+---------+---------+---------+---------+---------+-------+
only showing top 20 rows

###Data cleaning and rearrangement

In [102]:
#Retrieving the two columns from the existing tweet dataset (Data and tweet)
stock_tweet_df = trumpTweet_groupedDate[['date', 'content']]

In [103]:
stock_tweet_df['date'] =pd.to_datetime(stock_tweet_df.date)

/local_disk0/tmp/1588207156282-0/PythonShell.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 from __future__ import absolute_import

In [104]:
#stock_tweet_df.sort_values(by = 'date')
stock_tweet_df.head()

,date,content
0,2018-01-01,HAPPY NEW YEAR! We are MAKING AMERICA GREAT AG...
1,2019-01-01,HAPPY NEW YEAR!pic.twitter.com/bHoPDPQ7G6-MEXI...
2,2020-01-01,"Get this straightened out, Governor @ GavinNew..."
3,2018-01-10,"Today, it was my great honor to sign a new Exe..."
4,2019-01-10,The Mainstream Media has NEVER been more disho...


In [105]:
Topics_df = df_document_topic[0:]

In [106]:
#Joining the two dataframes (tweets and topics)
Topic_stock_tweet_df = pd.concat([stock_tweet_df, df_document_topic], axis=1)

In [107]:
Topic_stock_tweet_df

,date,content,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
0,2018-01-01,HAPPY NEW YEAR! We are MAKING AMERICA GREAT AG...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,9
1,2019-01-01,HAPPY NEW YEAR!pic.twitter.com/bHoPDPQ7G6-MEXI...,0.29,0.00,0.00,0.00,0.00,0.00,0.00,0.16,0.00,0.55,9
2,2020-01-01,"Get this straightened out, Governor @ GavinNew...",0.61,0.00,0.00,0.00,0.00,0.38,0.00,0.00,0.00,0.00,0
3,2018-01-10,"Today, it was my great honor to sign a new Exe...",0.00,0.00,0.73,0.00,0.00,0.00,0.00,0.00,0.00,0.26,2
4,2019-01-10,The Mainstream Media has NEVER been more disho...,0.00,0.00,0.00,0.00,0.13,0.00,0.00,0.00,0.00,0.86,9
5,2020-01-10,"THANK YOU TOLEDO, OHIO!-Under my administratio...",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,0.00,8
6,2018-01-11,The United States needs the security of the Wa...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.22,0.00,0.77,9
7,2019-01-11,Dear Diary...-Will be interviewed at the Borde...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,9
8,2020-01-11,Will be interviewed tonight by Laura @ Ingraha...,0.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
9,2018-01-12,"Yesterday, I signed the # INTERDICTAct (H.R. 2...",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,9


In [108]:
Topic_stock_tweet_df = Topic_stock_tweet_df.sort_values(by = 'date')

In [109]:
national_grid_pd = national_grid_stock_df.toPandas()

In [110]:
Topic_stock_tweet_df["Prices"] = ""

In [111]:
indx=0
for i in range (0,len(Topic_stock_tweet_df)):
    for j in range (0,len(national_grid_pd)):
        get_tweet_date=Topic_stock_tweet_df.date.iloc[i]
        get_stock_date=national_grid_pd.Date.iloc[j]
        if(get_stock_date==get_tweet_date):
            print(get_stock_date," ",get_tweet_date)
            Topic_stock_tweet_df.at(i,'Prices',national_grid_pd.Close[j])
            break

In [112]:
national_grid_pd['Date'] =pd.to_datetime(national_grid_pd.Date)
#type(national_grid_pd['Date'])

In [113]:
#Joining the Tweet_topic dataset with the stock price Dataset
Topic_stock_df = pd.merge(left=Topic_stock_tweet_df, left_on='date',
         right=national_grid_pd, right_on='Date', how = 'left')

In [114]:
Topic_stock_df.head()

,date,content,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic,Prices,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-20,Thank you for joining us at the Lincoln Memori...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,9,,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-21,THANK YOU for another wonderful evening in Was...,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.87,9,,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-22,Had a great meeting at CIA Headquarters yester...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.98,9,,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-23,Busy week planned with a heavy focus on jobs a...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.94,9,,2017-01-23,63.810043,64.159386,63.722706,64.104805,54.674591,459000
4,2017-01-24,Will be meeting at 9:00 with top automobile ex...,0.00,0.98,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,,2017-01-24,63.799126,64.181221,63.777294,63.897381,54.497673,652000


In [115]:
Topic_stock_df_clean = Topic_stock_df.drop(Topic_stock_df.ix[:, 'Prices':'Low'].columns, axis = 1)

/local_disk0/tmp/1588207156282-0/PythonShell.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
 from __future__ import absolute_import
/databricks/python/lib/python3.7/site-packages/pandas/core/indexing.py:888: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
 retval = getattr(retval, self.name)._getitem_axis(key, axis=i)

In [116]:
Topic_stock_df_clean = Topic_stock_df_clean.drop(columns = ['Adj Close', 'Volume'], axis = 1)

In [117]:
Topic_stock_df_clean.rename(columns={'Close': 'Prices'}, inplace=True)
Topic_stock_df_clean.head()

,date,content,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic,Prices
0,2017-01-20,Thank you for joining us at the Lincoln Memori...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,9,NaN
1,2017-01-21,THANK YOU for another wonderful evening in Was...,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.87,9,NaN
2,2017-01-22,Had a great meeting at CIA Headquarters yester...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.98,9,NaN
3,2017-01-23,Busy week planned with a heavy focus on jobs a...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.94,9,64.104805
4,2017-01-24,Will be meeting at 9:00 with top automobile ex...,0.00,0.98,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,63.897381


####Filling the empty stock price cells

In [119]:
#Forward filling the cells with stock price values i.e. Weekends are filled with Friday's values
Topic_stock_df_clean["Prices"].fillna( method ='ffill', inplace = True) 

In [120]:
Topic_stock_df_clean['Prices'] = Topic_stock_df_clean['Prices'].astype(float)

In [121]:
#Filling the rest of the empty cells with the mean value of the prices
Topic_stock_df_clean["Prices"].fillna(Topic_stock_df_clean.mean()['Prices'], inplace = True)

In [122]:
Topic_stock_df_clean["label"] = Topic_stock_df_clean["Prices"]

In [123]:
Topic_stock_df_clean.head()

,date,content,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic,Prices,label
0,2017-01-20,Thank you for joining us at the Lincoln Memori...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,9,58.154025,58.154025
1,2017-01-21,THANK YOU for another wonderful evening in Was...,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.87,9,58.154025,58.154025
2,2017-01-22,Had a great meeting at CIA Headquarters yester...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.98,9,58.154025,58.154025
3,2017-01-23,Busy week planned with a heavy focus on jobs a...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.94,9,64.104805,64.104805
4,2017-01-24,Will be meeting at 9:00 with top automobile ex...,0.00,0.98,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,63.897381,63.897381


In [124]:
#Verifying skewness of the data
Topic_stock_df_clean.Prices.plot.hist(bins=15, alpha=0.5)

In [125]:
Topic_stock_df_clean.Prices.mean()

Out[225]: 58.15402514061137

In [126]:
Topic_stock_df_clean.Prices.median()

Out[226]: 57.220001

In [127]:
Topic_stock_df_spark = spark.createDataFrame(Topic_stock_df_clean)

###Supervised Learning model for Price Prediction

In [129]:
#Random Forest Regressor

from pyspark.ml.feature import VectorAssembler

feature_list = []
for col in Topic_stock_df_clean.columns:
    if col == 'label' or col == 'Prices' or col == 'date' or col == 'content' or col == 'dominant_topic' or col == 'log_value' :
        continue
    else:
        feature_list.append(col)

assembler = VectorAssembler(inputCols=feature_list, outputCol="features")
print(feature_list)

['Topic0', 'Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9']

In [130]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(labelCol="label", featuresCol="features")

In [131]:
#Pipline for assembler and randomforest regressor
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[assembler, rf])

In [132]:
#Configuring Grid Search Parameters
from pyspark.ml.tuning import ParamGridBuilder
import numpy as np

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [int(x) for x in np.linspace(start = 25, stop = 35, num = 5)]) \
    .addGrid(rf.maxDepth, [int(x) for x in np.linspace(start = 6, stop = 12, num = 5)]) \
    .addGrid(rf.featureSubsetStrategy, ['auto', 'sqrt']) \
    .build()

In [133]:
#Configuring Crossvalidator with RandomForest Pipeline, Grid Searchm and Regression Performance evaluator to get the best model
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(),
                          numFolds=3)

In [134]:
%sh pip install mlflow

Requirement already satisfied: mlflow in /databricks/python3/lib/python3.7/site-packages (1.8.0)
Requirement already satisfied: python-dateutil in /databricks/python3/lib/python3.7/site-packages (from mlflow) (2.8.0)
Requirement already satisfied: pyyaml in /databricks/python3/lib/python3.7/site-packages (from mlflow) (5.3.1)
Requirement already satisfied: alembic in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.4.2)
Requirement already satisfied: requests>=2.17.3 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (2.21.0)
Requirement already satisfied: Flask in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.1.2)
Requirement already satisfied: sqlparse in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.3.1)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.24.2)
Requirement already satisfied: click>=7.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (7.1.2)
Requirement already satisfied: cloudpickle in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.4.1)
Requirement already satisfied: sqlalchemy<=1.3.13 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.3.13)
Requirement already satisfied: six>=1.10.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.12.0)
Requirement already satisfied: docker>=4.0.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (4.2.0)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.16.2)
Requirement already satisfied: gorilla in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.3.0)
Requirement already satisfied: gitpython>=2.1.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (3.1.1)
Requirement already satisfied: querystring-parser in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.2.4)
Requirement already satisfied: simplejson in /databricks/python3/lib/python3.7/site-packages (from mlflow) (3.17.0)
Requirement already satisfied: gunicorn; platform_system != "Windows" in /databricks/python3/lib/python3.7/site-packages (from mlflow) (20.0.4)
Requirement already satisfied: databricks-cli>=0.8.7 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.10.0)
Requirement already satisfied: entrypoints in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.3)
Requirement already satisfied: prometheus-flask-exporter in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.13.0)
Requirement already satisfied: protobuf>=3.6.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (3.11.3)
Requirement already satisfied: Mako in /databricks/python3/lib/python3.7/site-packages (from alembic->mlflow) (1.1.2)
Requirement already satisfied: python-editor>=0.3 in /databricks/python3/lib/python3.7/site-packages (from alembic->mlflow) (1.0.4)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests>=2.17.3->mlflow) (2019.3.9)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests>=2.17.3->mlflow) (2.8)
Requirement already satisfied: urllib3<1.25,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests>=2.17.3->mlflow) (1.24.1)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /databricks/python3/lib/python3.7/site-packages (from requests>=2.17.3->mlflow) (3.0.4)
Requirement already satisfied: itsdangerous>=0.24 in /databricks/python3/lib/python3.7/site-packages (from Flask->mlflow) (1.1.0)
Requirement already satisfied: Jinja2>=2.10.1 in /databricks/python3/lib/python3.7/site-packages (from Flask->mlflow) (2.11.2)
Requirement already satisfied: Werkzeug>=0.15 in /databricks/python3/lib/python3.7/site-packages (from Flask->mlflow) (1.0.1)
Requirement already satisfied: pytz>=2011k in /databricks/python3/lib/python3.7/site-package

In [135]:
(trainingData, testData) = Topic_stock_df_spark.randomSplit([0.8, 0.2])

In [136]:
#Training the model
cvModel = crossval.fit(trainingData)


/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [137]:
#Error calculation on the dataset by the best model
predictions = cvModel.transform(testData)
rmse_testData = evaluator.evaluate(predictions)
print("The rmse after test is %f" %rmse_testData)

The rmse after test is 5.605425

In [138]:
best_model = cvModel.bestModel.stages[-1]

In [139]:
type(best_model)

Out[238]: pyspark.ml.regression.RandomForestRegressionModel

In [140]:
type(cvModel)

Out[239]: pyspark.ml.tuning.CrossValidatorModel

In [141]:
display(predictions.toPandas().head())

date,content,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic,Prices,label,features,prediction
2017-01-28T00:00:00.000+0000,"I promise that our administration will ALWAYS have your back. We will ALWAYS be with you!pic.twitter.com/D0aOWhOH4X-The failing @ nytimes has been wrong about me from the very beginning. Said I would lose the primaries, then the general election. FAKE NEWS!-Thr coverage about me in the @ nytimes and the @ washingtonpost gas been so false and angry that the times actually apologized to its.....-...dwindling subscribers and readers.They got me wrong right from the beginning and still have not changed course, and never will. DISHONEST-Today, we remember the crew of the Space Shuttle Challenger, 31 years later. # NeverForgetpic.twitter.com/OhshQsFRfl",0.0,0.0,0.0,0.17,0.0,0.0,0.0,0.0,0.0,0.82,9,63.187775,63.187775,"List(0, 10, List(3, 9), List(0.17, 0.82))",56.866608537654045
2017-02-07T00:00:00.000+0000,"An extended interview from the Super Bowl with @ oreillyfactor airs tonight at 8:00 P.M. Enjoy!pic.twitter.com/kZdHqaNTVR-The failing @ nytimes was forced to apologize to its subscribers for the poor reporting it did on my election win. Now they are worse!-The threat from radical Islamic terrorism is very real, just look at what is happening in Europe and the Middle-East. Courts must act fast!-I don't know Putin, have no deals in Russia, and the haters are going crazy - yet Obama can make a deal with Iran, #1 in terror, no problem!-An honor having the National Sheriffs' Assoc. join me at the @ WhiteHouse. Incredible men & women who protect & serve 24/7/365. THANK YOU!!pic.twitter.com/9EMTnH0OrF",0.0,0.0,0.51,0.0,0.0,0.0,0.0,0.0,0.0,0.48,2,64.323143,64.323143,"List(0, 10, List(2, 9), List(0.51, 0.48))",56.85758360329523
2017-02-10T00:00:00.000+0000,"SEE YOU IN COURT, THE SECURITY OF OUR NATION IS AT STAKE!-LAWFARE: ""Remarkably, in the entire opinion, the panel did not bother even to cite this (the) statute."" A disgraceful decision!-The failing @ nytimes does major FAKE NEWS China story saying ""Mr.Xi has not spoken to Mr. Trump since Nov.14."" We spoke at length yesterday!",0.0,0.0,0.0,0.0,0.27,0.0,0.0,0.0,0.0,0.71,9,65.371178,65.371178,"List(0, 10, List(4, 9), List(0.27, 0.71))",58.953257715563865
2017-02-13T00:00:00.000+0000,Today I will meet with Canadian PM Trudeau and a group of leading business women to discuss women in the workforce. -Welcome to the @ WhiteHouse Prime Minister @ JustinTrudeau!pic.twitter.com/WKgF8Zo9ri-Wonderful meeting with Canadian PM @ JustinTrudeau and a group of leading CEO's & business women from Canadaand the United Statespic.twitter.com/Rxr31QpxMK,0.33,0.64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,65.382095,65.382095,"List(0, 10, List(0, 1), List(0.33, 0.64))",58.30448101976918
2017-02-23T00:00:00.000+0000,'S&P 500 Edges Higher After Trump Renews Jobs Pledge',0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.89,0.01,8,66.539299,66.539299,"List(1, 10, List(), List(0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.89, 0.01))",63.25728396424427


In [142]:
rmse

Out[267]: 5.20397209894708

##Visualizing the error and actual price vs predicted prices

In [144]:
import matplotlib.pyplot as plt

evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(predictions)

rfPred = cvModel.transform(Topic_stock_df_spark)
rmse = evaluator.evaluate(rfPred)
rfResult = rfPred.toPandas()

plt.plot(rfResult.label, rfResult.prediction, 'bo')
plt.xlabel('Price')
plt.ylabel('Prediction')
plt.suptitle("Model Performance RMSE: %f" % rmse)
plt.show()

In [145]:
plt.figure(figsize=(10, 8))
plt.plot(rfResult['date'], rfResult['Prices'], 'b-', label = 'Actual Prices')
plt.plot(rfResult['date'], rfResult['prediction'], 'r-', label = 'Predicted Prices')
plt.xlabel('Date'); plt.ylabel('Stock Prices ($)'); plt.title('National Grid Energy Industry')
plt.legend();

##Final Implementation of Predicting Prices from Tweets

In [147]:
# Define function to predict topic for a given text document.
nlp = spacy.load('en', disable=['parser', 'ner'])

def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization

    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))

    # Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Step 3: Vectorize transform
    mytext_4 = countVectorizer.transform(mytext_3)

    # Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

# Predict the topic
mytext = ["In light of the attack from the Invisible Enemy, as well as the need to protect the jobs of our GREAT American Citizens, I will be signing an Executive Order to temporarily suspend immigration into the United States!"]
topic, prob_scores = predict_topic(text = mytext)
NewStock_df = pd.DataFrame(data=prob_scores, columns=["Topic0", "Topic1", "Topic2", "Topic3", "Topic4", "Topic5", "Topic6", "Topic7", "Topic8", "Topic9"])
print(topic)
print(prob_scores)
NewStock_df_spark = spark.createDataFrame(NewStock_df)
StockPred = cvModel.transform(NewStock_df_spark)
print("The predicted stock price of the Energy market is predicted to be : %f" %StockPred.toPandas().prediction)

['great', 'peopl', 'democrat', 'fake', 'news', 'presid', 'countri', 'trump', 'year', 'time', 'want', 'get', 'us', 'border', 'big']
[[0.0055572 0.00555756 0.09781995 0.00555556 0.00555565 0.0055556
 0.00555668 0.00555704 0.36982184 0.49346293]]
/databricks/spark/python/pyspark/sql/dataframe.py:2163: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)
The predicted stock price of the Energy market is predicted to be : 62.526227